In [3]:
# %% [markdown]
# # Phyllis Schlafly — Page‑by‑Page OCR Extract
# 
# This notebook‑style Python script (compatible with **Jupyter** via Jupytext)
# demonstrates how to call `page_text_extractor.extract_pages()` on a single
# month‑PDF, save the raw text of every page, and preview the results.
# 
# **How to use:**
# 1. Open this file in JupyterLab / VS Code with the *Python* notebook renderer.
# 2. Edit the configuration variables in **Cell 2**.
# 3. Execute cells top‑to‑bottom.
# 4. Inspect the generated `.txt` files or the Pandas preview.
# 
# ---
# 
# *Generated by ChatGPT (o3) – 2025‑04‑22*

# %% [markdown]
# ## 1 · Imports
# Standard setup: numpy/pandas for inspection plus the extractor helper.

# %%
from pathlib import Path
import pandas as pd
from page_text_extractor import extract_pages, ExtractConfig

# %% [markdown]
# ## 2 · Run Parameters
# Update these paths to point at the month you want to process.

# %%
PDF_PATH = Path(
    "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/data/raw/comm/2002/01.pdf"
)
OUT_DIR = Path(
    "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/data/ocr/2002/01"
)

config = ExtractConfig(
    pdf_path=PDF_PATH,
    out_dir=OUT_DIR,
    dpi=300,
    lang="eng",
    overwrite=False,  # set True to force re‑OCR
    show_progress=True,
)
print(config)

# %% [markdown]
# ## 3 · Run Extraction

# %%
extract_pages(config)

# %% [markdown]
# ## 4 · Quick Preview
# Load the first few page‑texts into a DataFrame to skim for obvious OCR issues.

# %%
page_files = sorted(OUT_DIR.glob("page_*.txt"))
rows = []
for f in page_files[:5]:
    with f.open("r", encoding="utf‑8") as fh:
        text = fh.read()
    rows.append({"file": f.name, "snippet": text[:300].replace("\n", " ") + "…"})

df_preview = pd.DataFrame(rows)
df_preview

# %% [markdown]
# ---
# 🎉 **Done.** You now have one `.txt` file per page in `OUT_DIR`.  Continue
# refining the pipeline or exploring OCR quality as needed.


ExtractConfig(pdf_path=PosixPath('/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/data/raw/comm/2002/01.pdf'), out_dir=PosixPath('/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/data/ocr/2002/01'), dpi=300, lang='eng', overwrite=False, show_progress=True)
[INFO] Opening /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/data/raw/comm/2002/01.pdf


OCR pages: 100%|██████████| 48/48 [02:15<00:00,  2.82s/page]

[INFO] Wrote 1 text files to /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/data/ocr/2002/01


,file,snippet
0,page_000.txt,Phyllis Schlafly Commentaries on Christian Rad...
1,page_001.txt,…
2,page_002.txt,New Year’s Resolutions for Border Reform On t...
3,page_003.txt,…
4,page_004.txt,~ e . : O20 Security Starts at Our Borders WR...
